In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import torchaudio
import soundfile as sf
import seaborn as sns

from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import Audio

from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from birdnetlib.batch import DirectoryMultiProcessingAnalyzer

In [3]:
train_dir = Path('E:\data\BirdCLEF')

class CFG:
    random_seed = 42
    
    ROOT_FOLDER = train_dir
    AUDIO_FOLDER = train_dir / 'train_audio'
    DATA_DIR = train_dir / 'spectros'
    TRAIN_CSV = train_dir / 'train_metadata.csv'
    RESULTS_DIR = train_dir / 'results'
    CKPT_DIR = RESULTS_DIR / 'ckpt'
    bird20223 = train_dir / 'bird2023.csv'
    UNLABELED_FOLDER = train_dir / 'unlabeled_soundscapes'

In [4]:
meta_df = pd.read_csv(CFG.TRAIN_CSV)
df_23 = pd.read_csv(CFG.bird20223)
df_23.shape, df_23.shape

((16941, 12), (16941, 12))

In [5]:
directories = meta_df.primary_label.unique().tolist()
directories = [str(CFG.AUDIO_FOLDER / d) for d in directories]
len(directories), directories[0]

(182, 'E:\\data\\BirdCLEF\\train_audio\\asbfly')

In [6]:
meta_df['filename'] = f'{str(CFG.AUDIO_FOLDER)}\\' + meta_df['filename']

In [7]:
meta_df.head(2)

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,E:\data\BirdCLEF\train_audio\asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,E:\data\BirdCLEF\train_audio\asbfly/XC164848.ogg


In [8]:
prim_df = meta_df[meta_df['secondary_labels'] == '[]']
prim_df.shape

(22567, 12)

In [9]:
meta_df[meta_df['primary_label'] == 'zitcis1']

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
23959,zitcis1,['zitcis1'],['call'],44.6469,-1.1200,Cisticola juncidis,Zitting Cisticola,Mathieu Taillade,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/124995,E:\data\BirdCLEF\train_audio\zitcis1/XC124995.ogg
23960,zitcis1,[],['song'],13.0373,104.4865,Cisticola juncidis,Zitting Cisticola,Patrik Åberg,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/125068,E:\data\BirdCLEF\train_audio\zitcis1/XC125068.ogg
23961,zitcis1,['bkwsti'],"['call', 'song']",32.7152,-9.0598,Cisticola juncidis,Zitting Cisticola,Marcin Sołowiej,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/125875,E:\data\BirdCLEF\train_audio\zitcis1/XC125875.ogg
23962,zitcis1,[],['song'],-28.8865,24.8353,Cisticola juncidis,Zitting Cisticola,Sander Bot,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/126356,E:\data\BirdCLEF\train_audio\zitcis1/XC126356.ogg
23963,zitcis1,[],['song'],-28.8865,24.8353,Cisticola juncidis,Zitting Cisticola,Sander Bot,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/126357,E:\data\BirdCLEF\train_audio\zitcis1/XC126357.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...
24454,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/845747,E:\data\BirdCLEF\train_audio\zitcis1/XC845747.ogg
24455,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/845817,E:\data\BirdCLEF\train_audio\zitcis1/XC845817.ogg
24456,zitcis1,[],[''],51.1207,4.5607,Cisticola juncidis,Zitting Cisticola,Wim Jacobs,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/856176,E:\data\BirdCLEF\train_audio\zitcis1/XC856176.ogg
24457,zitcis1,[],[''],41.5607,-8.4236,Cisticola juncidis,Zitting Cisticola,Jorge Leitão,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://xeno-canto.org/856723,E:\data\BirdCLEF\train_audio\zitcis1/XC856723.ogg


### Analyze

In [10]:
prim_df.iloc[0]

primary_label                                                  asbfly
secondary_labels                                                   []
type                                                         ['call']
latitude                                                      39.2297
longitude                                                    118.1987
scientific_name                                    Muscicapa dauurica
common_name                                    Asian Brown Flycatcher
author                                                 Matt Slaymaker
license             Creative Commons Attribution-NonCommercial-Sha...
rating                                                            5.0
url                                 https://www.xeno-canto.org/134896
filename             E:\data\BirdCLEF\train_audio\asbfly/XC134896.ogg
Name: 0, dtype: object

In [11]:
filename = prim_df.iloc[0].filename
filename = r"E:\data\BirdCLEF\unlabeled_soundscapes\1225526.ogg"
filename

'E:\\data\\BirdCLEF\\unlabeled_soundscapes\\1225526.ogg'

In [17]:
# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()

recording = Recording(
    analyzer,
    filename,
    # lat=35.4244,
    # lon=-120.7463,
    # date=datetime(year=2022, month=5, day=10), # use date or week_48
    overlap=2,
    min_conf=0.6,
)
recording.analyze()
len(recording.detections)

Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
read_audio_data
read_audio_data: complete, read  239 chunks.
analyze_recording 1225526.ogg


9

In [18]:
recording.detections

[{'common_name': 'American Crow',
  'scientific_name': 'Corvus brachyrhynchos',
  'start_time': 32.0,
  'end_time': 35.0,
  'confidence': 0.7973770499229431,
  'label': 'Corvus brachyrhynchos_American Crow'},
 {'common_name': 'American Crow',
  'scientific_name': 'Corvus brachyrhynchos',
  'start_time': 59.0,
  'end_time': 62.0,
  'confidence': 0.8308204412460327,
  'label': 'Corvus brachyrhynchos_American Crow'},
 {'common_name': 'American Crow',
  'scientific_name': 'Corvus brachyrhynchos',
  'start_time': 60.0,
  'end_time': 63.0,
  'confidence': 0.6368460655212402,
  'label': 'Corvus brachyrhynchos_American Crow'},
 {'common_name': 'American Crow',
  'scientific_name': 'Corvus brachyrhynchos',
  'start_time': 61.0,
  'end_time': 64.0,
  'confidence': 0.9614817500114441,
  'label': 'Corvus brachyrhynchos_American Crow'},
 {'common_name': 'American Crow',
  'scientific_name': 'Corvus brachyrhynchos',
  'start_time': 63.0,
  'end_time': 66.0,
  'confidence': 0.6711816191673279,
  'lab

In [14]:
recording.path, recording.path.split('/')[0].split('\\')[-1]

('E:\\data\\BirdCLEF\\unlabeled_soundscapes\\1225526.ogg', '1225526.ogg')

In [15]:
label = recording.path.split('/')[0].split('\\')[-1]

# for det in recording.detections:
#     print(det['start_time'])
data = [(label, x['start_time'], x['end_time']) for x in recording.detections]

data[0]

('1225526.ogg', 60.0, 63.0)

In [16]:
recording.duration

240.0

In [15]:
def on_analyze_directory_complete(recordings, file=train_dir / "bird_preds.csv"):
    preds = []
    
    for rec in recordings:
        if rec.error:
            print(f'{rec.error_message} in {rec.path}')
        else:
            filename= rec.path.split('\\')[-1]
            label = rec.path.split('\\')[-2]
            
            # print(filename, label)
            duration = rec.duration
            
            data = [(filename, label, x['scientific_name'], x['start_time'], x['end_time'], duration, x['confidence']) for x in rec.detections]
            preds.append(pd.DataFrame(data, columns = ['filename', 'label', 'name', 'start', 'end', 'duration', 'confidence']))

    print(len(preds))

    results_df = pd.concat(preds, axis=0)

    results_df.to_csv(file, index=False)    
    
    # return preds

In [16]:
directory = directories[1]
directory = CFG.AUDIO_FOLDER
print(directory)

batch = DirectoryMultiProcessingAnalyzer(
    directory,
    analyzers=[analyzer],
    patterns=["*/*.ogg"],
    # lon=-120.7463,
    # lat=35.4244,
    # # date=datetime(year=2022, month=5, day=10),
    min_conf=0.8,
)

batch.on_analyze_directory_complete = on_analyze_directory_complete

E:\data\BirdCLEF\train_audio


In [17]:
batch.process()

24459


C:\Users\Asus\AppData\Local\Temp\ipykernel_2556\1730626807.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat(preds, axis=0)


### Predict unlabeled

In [18]:
def unlabeled_complete(recordings, file= train_dir / "unlabeled_preds.csv"):
    preds = []
    
    for rec in recordings:
        if rec.error:
            print(f'{rec.error_message} in {rec.path}')
        else:
            filename = rec.path.split('\\')[-1]
            
            # print(filename)
            print(len(rec.detections))
            
            data = [(filename, x['scientific_name'], x['start_time'], x['end_time'], x['confidence']) for x in rec.detections]
            preds.append(pd.DataFrame(data, columns = ['filename', 'name', 'start', 'end', 'confidence']))


    print(len(preds))

    results_df = pd.concat(preds, axis=0)

    results_df.to_csv(file, index=False)  

In [19]:
directory = CFG.UNLABELED_FOLDER

batch = DirectoryMultiProcessingAnalyzer(
    directory,
    analyzers=[analyzer],
    patterns=["*.ogg"],
    # patterns=["460830.ogg"],
    # lon=10.1667,
    # lat=77.0667,
    # # date=datetime(year=2022, month=5, day=10),
    min_conf=0.75,
)

batch.on_analyze_directory_complete = unlabeled_complete

In [20]:
batch.process()

0
0
0
2
0
0
2
10
0
0
0
0
0
10
12
0
4
5
17
2
0
0
0
0
10
0
14
0
1
7
0
0
0
0
1
3
0
5
1
0
0
24
0
2
1
26
0
20
0
0
0
12
1
0
0
2
0
0
0
1
9
0
7
0
2
7
0
0
0
0
4
0
1
0
0
0
0
1
16
1
0
0
2
0
0
2
0
0
0
0
5
0
0
0
1
0
0
12
1
0
2
0
0
0
0
0
0
0
8
41
0
0
4
0
0
0
7
0
1
0
12
1
23
0
1
0
1
0
2
0
0
4
0
1
0
1
1
1
4
0
2
4
2
2
0
0
0
13
0
0
2
0
0
0
0
0
1
2
0
3
0
0
0
0
6
23
0
0
0
0
0
1
1
0
0
1
0
0
3
0
1
2
0
0
4
7
15
1
11
0
10
0
0
0
0
12
0
0
0
0
0
3
3
33
1
3
0
1
0
0
2
0
0
0
2
0
0
0
0
0
0
0
0
1
0
16
0
3
0
0
0
14
0
6
0
1
0
0
0
0
0
0
11
0
0
0
1
0
0
0
0
0
0
0
1
1
0
1
2
0
9
5
0
0
0
0
3
0
2
0
0
2
8
1
1
0
0
0
0
0
0
0
19
0
0
0
0
17
3
0
10
0
0
0
3
0
5
2
11
1
25
0
3
19
3
0
0
0
1
1
12
0
0
6
25
18
0
1
0
0
0
6
18
2
0
7
1
0
0
0
0
1
0
4
0
0
0
2
3
0
1
0
11
0
3
5
0
2
17
0
18
0
0
1
0
0
14
0
2
0
0
0
17
2
0
18
2
6
0
10
0
4
1
1
0
3
0
2
0
0
0
2
0
0
1
0
0
1
0
0
7
0
19
0
0
0
3
0
5
0
11
18
1
2
0
35
0
2
0
0
4
0
0
0
0
1
0
2
0
13
0
0
9
0
0
0
9
0
0
0
0
0
0
0
0
0
25
2
0
26
27
0
0
0
6
1
0
19
0
0
0
0
14
0
3
1
0
0
23
2
0
0
0
2
1
0
0
2
6
0
0
5
0
0

C:\Users\Asus\AppData\Local\Temp\ipykernel_2556\1805713660.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat(preds, axis=0)
